In [1]:
!pip install datasets
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pickle
from datasets import load_dataset

In [3]:
# Load the Rotten Tomatoes dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

train_text = train_dataset.to_pandas()['text']

max_text_len = 0
for text in train_text:
    max_text_len = max(max_text_len, len(text))

print(max_text_len)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

267


In [4]:
print(train_dataset[0])

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


In [5]:
# Load the embedding matrix and vocab from files
with open('embedding_matrix_300d.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f).astype(np.float32)
    padding = [0 for i in range(300)]
    embedding_matrix = np.insert(embedding_matrix, 0, padding, 0)
    print(type(embedding_matrix))

with open('vocab_word_to_index_300d.pkl', 'rb') as f:
    vocab_word_to_index = pickle.load(f)
    print(type(vocab_word_to_index))

# Convert to torch tensors
embedding_matrix = torch.tensor(embedding_matrix)
vocab_size, embedding_dim = embedding_matrix.shape
print(embedding_matrix[0])

<class 'numpy.ndarray'>
<class 'dict'>
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [8]:
import numpy as np
import torch
import nltk
from torch.utils.data import TensorDataset, DataLoader

class SentimentDataset:
    def __init__(self, dataset, word_to_index, max_len=30):
        self.dataset = dataset
        self.word_to_index = word_to_index
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['text']
        label = self.dataset[idx]['label']

        # Tokenization and word-to-index conversion
        text = text.lower()
        word_list = nltk.tokenize.word_tokenize(text)
        word_list = [word.strip("'\"") for word in word_list]
        indices = [self.word_to_index.get(word, self.word_to_index.get('<UNK>')) + 1 for word in word_list]
        indices = indices[:self.max_len] + [0] * (self.max_len - len(indices))  # Padding

        return np.array(indices), np.array(label)

    def preprocess_data(self):
        texts = []
        labels = []
        for i in range(len(self.dataset)):
            features, label = self.__getitem__(i)
            texts.append(features)
            labels.append(label)
        return np.array(texts), np.array(labels)

train_texts, train_labels = SentimentDataset(train_dataset, vocab_word_to_index, max_len=30).preprocess_data()
valid_texts, valid_labels = SentimentDataset(valid_dataset, vocab_word_to_index, max_len=30).preprocess_data()
test_texts, test_labels = SentimentDataset(test_dataset, vocab_word_to_index, max_len=30).preprocess_data()

# Convert preprocessed arrays to PyTorch tensors
train_texts = torch.tensor(train_texts)
train_labels = torch.tensor(train_labels)
valid_texts = torch.tensor(valid_texts)
valid_labels = torch.tensor(valid_labels)
test_texts = torch.tensor(test_texts)
test_labels = torch.tensor(test_labels)

train_dataset = TensorDataset(train_texts, train_labels)
valid_dataset = TensorDataset(valid_texts, valid_labels)
test_dataset = TensorDataset(test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, num_layers = 1, freeze_embeddings=True, dropout = 0.5):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=freeze_embeddings)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True, device= device)
        #self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim, device= device)
        self.sigmoid = nn.Sigmoid()
        self.device = device


    def forward(self, x):
        embedded = self.embedding(x)
        out, hidden = self.rnn(embedded)
        out = out[:, -1, :]
        #out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

# Model hyperparameters
hidden_dim = 128
output_dim = 1  # Binary classification (positive, negative)

model = SentimentRNN(embedding_matrix, hidden_dim, output_dim, 1)
print(model.embedding.weight[1])

tensor([ 3.4451e-01, -6.5574e-04,  6.2947e-02, -1.2662e-01, -4.9770e-01,
        -1.0601e-01, -5.0291e-01, -2.8215e-01, -2.8976e-01,  7.3739e-01,
         4.0199e-01, -6.5565e-01,  2.8014e-01,  1.0140e-01, -5.4716e-01,
         7.2961e-02,  2.8495e-01,  2.4297e-01,  5.5845e-02, -1.0398e-01,
         5.3156e-01, -2.9247e-01, -6.9752e-01, -2.7535e-01, -5.4427e-02,
        -2.8416e-01,  3.0716e-01, -5.9556e-02,  3.7306e-02,  2.3405e-01,
        -5.2893e-01, -5.3697e-01,  1.2916e-01, -1.1276e-01,  7.0001e-01,
         3.2547e-02, -2.6703e-01,  3.6086e-01,  9.1463e-02,  1.7362e-01,
         5.4645e-01, -8.9706e-03, -6.4165e-01,  1.5314e-01,  8.3316e-03,
         3.2148e-02, -5.9214e-02,  1.6797e-01, -3.2526e-01, -2.2324e-01,
        -5.0746e-01,  1.2157e-01,  1.4054e-01,  6.3638e-01, -4.1387e-01,
        -4.5616e-01,  3.0049e-01,  8.0421e-02, -6.7546e-01, -3.2467e-02,
        -6.2736e-01, -5.0160e-01,  2.3964e-01,  2.8816e-01, -6.9562e-01,
         5.5617e-01,  2.0428e-01, -4.4790e-02,  5.0

In [10]:
# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# Move model to GPU if available
model.to(device)

patience = 5
best_metric = None
no_improvement_count = 0

# Training function
def train_model(model, train_loader, valid_loader, epochs=30):
    global best_metric, no_improvement_count #variables for early stopping
    for epoch in range(epochs):
        # Set model to training mode
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)

            optimizer.zero_grad()  # Reset gradients

            # Forward pass: get predictions
            predictions = model(texts)

            # Compute the loss
            loss = criterion(predictions, labels.unsqueeze(1).float())
            total_train_loss += loss.item()

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

            # Accuracy calculation
            predicted_labels = (predictions > 0.5).int()
            total_train_correct += (predicted_labels.squeeze() == labels).sum().item()

            #print(total_train_correct)
            total_train_samples += labels.size(0)

        # Calculate and print average training accuracy and loss per epoch
        train_accuracy = total_train_correct / total_train_samples
        train_loss = total_train_loss / len(train_loader)

        # Evaluate model on validation set
        valid_accuracy = evaluate_model(model, valid_loader)

        print(f'Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | Validation Accuracy: {valid_accuracy:.4f}')

        # Early stopping check
        if best_metric is None or valid_accuracy > best_metric:
            best_metric = valid_accuracy
            no_improvement_count = 0  # Reset counter
            torch.save(model.state_dict(), 'best_model.pt')  # Save best model state
        else:
            no_improvement_count += 1  # Increment counter if no improvement

        if no_improvement_count >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break  # Exit training loop if no improvement for `patience` epochs

def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in loader:
            texts, labels = texts.to(device), labels.to(device)
            predictions = model(texts)
            predicted_labels = (predictions >= 0.5).int()
            correct += (predicted_labels.squeeze() == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [11]:
import random
random.seed(42)

np.random.seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Ensure deterministic behavior for cuDNN (CUDA)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [12]:
# Train the model
train_model(model, train_loader, valid_loader, epochs=30)

Epoch 1 | Train Loss: 0.6947 | Train Accuracy: 0.4989 | Validation Accuracy: 0.4934
Epoch 2 | Train Loss: 0.6914 | Train Accuracy: 0.5125 | Validation Accuracy: 0.5178
Epoch 3 | Train Loss: 0.6469 | Train Accuracy: 0.6227 | Validation Accuracy: 0.6970
Epoch 4 | Train Loss: 0.5607 | Train Accuracy: 0.7165 | Validation Accuracy: 0.7411
Epoch 5 | Train Loss: 0.5274 | Train Accuracy: 0.7479 | Validation Accuracy: 0.7373
Epoch 6 | Train Loss: 0.5102 | Train Accuracy: 0.7605 | Validation Accuracy: 0.7233
Epoch 7 | Train Loss: 0.4999 | Train Accuracy: 0.7661 | Validation Accuracy: 0.7373
Epoch 8 | Train Loss: 0.4947 | Train Accuracy: 0.7707 | Validation Accuracy: 0.7383
Epoch 9 | Train Loss: 0.4886 | Train Accuracy: 0.7735 | Validation Accuracy: 0.7280
Early stopping triggered after 9 epochs


In [13]:
# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Evaluate the model on the test set
test_acc = evaluate_model(model, test_loader)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7458


<ipython-input-13-bda9ce76bc51>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pt'))
